In [1]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import io
import matplotlib.pyplot as plt
import time
import os
import copy
from glob import glob
from tqdm import tqdm
import warnings
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score,recall_score,roc_auc_score,roc_curve,confusion_matrix
warnings.simplefilter('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.data import Subset
import albumentations as A
from PIL import Image
from torchvision.transforms import ToTensor, Normalize
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
from torch.utils.data import Dataset
import nibabel as nib
import albumentations as A
import pydicom
import cv2
from torchvision import transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.get_device_name())

cuda:0
NVIDIA GeForce RTX 3060 Laptop GPU


In [2]:
height,width = (500,250) 
class CustomDataset(Dataset):
    def __init__(self, img_paths=None, transform=None):
        self.img_paths = img_paths
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((height,width)),
            transforms.ToTensor()
        ])
        
        # self.to_tensor = ToTensor()
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        img=cv2.imread(self.img_paths[idx])
        img=img.astype('uint8')
        # img = self.to_tensor(img)
        img=self.transform(img)
        return img

In [3]:
batch_size = 1
num_workers = 0
img_paths=glob("/home/uraninjo/Desktop/Çalışmalar/Comp/SegmentasyonOutput/*/*.png")
dataset = CustomDataset(img_paths=img_paths)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
len(img_paths)

2000

In [ ]:
plt.imshow(np.transpose(dataset[2],(1,2,0)))

In [5]:
import torch

def test(birads_model, kompozisyon_model, dataloader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    birads_model.eval()
    kompozisyon_model.eval()
    # print("birads_model: ",birads_model)
    # print("kompozisyon_model: ",kompozisyon_model)
    with torch.no_grad():
        for i, inputs in enumerate(dataloader):
            inputs = inputs.to(device)
            birads_outputs = birads_model(inputs)
            birads_predicted = torch.argmax(birads_outputs)

            kompozisyon_outputs = kompozisyon_model(inputs)
            kompozisyon_predicted = torch.argmax(kompozisyon_outputs)
            
            # print(kompozisyon_outputs)
            # print(kompozisyon_predicted.cpu().numpy())
            # print("-----------")
            # print(birads_outputs)
            # print(birads_predicted.cpu().numpy())
            # print("******************************\n\n")
            if birads_predicted.cpu().numpy()== 0:
                birads_predicted_label = "BI-RADS0"
            elif birads_predicted.cpu().numpy()== 1:
                birads_predicted_label = "BI-RADS1-2"
            elif birads_predicted.cpu().numpy()== 2:
                birads_predicted_label = "BI-RADS4-5"


            if kompozisyon_predicted.cpu().numpy()== 0:
                kompozisyon_predicted_label = "A"
            elif kompozisyon_predicted.cpu().numpy()== 1:
                kompozisyon_predicted_label = "B"
            elif kompozisyon_predicted.cpu().numpy()== 2:
                kompozisyon_predicted_label = "C"
            elif kompozisyon_predicted.cpu().numpy()== 3:
                kompozisyon_predicted_label = "D"
            _tür=img_paths[i].split("/")[-1]
            _hno=img_paths[i].split("/")[-2]
            
            # plt.imshow(np.transpose(inputs[0].cpu().numpy(),(1,2,0)))
            # plt.text(0,0,str(birads_predicted_label)+" "+str(kompozisyon_predicted_label+ " "+str(_tür)+" "+str(_hno)))
            # print(birads_predicted_label)
            # print(kompozisyon_predicted_label)
            # print("******************************\n\n")
            # plt.pause(0.001)
            
            
            try:
                os.mkdir(f"/home/uraninjo/Desktop/Çalışmalar/Comp/LABELS")
            except:
                print("LABELS dosyası zaten var")
            try:
                os.mkdir(f"/home/uraninjo/Desktop/Çalışmalar/Comp/LABELS/{_hno}")
            except:
                print("txt dosyası zaten var")

            if os.path.exists(f"/home/uraninjo/Desktop/Çalışmalar/Comp/LABELS/{_hno}/results.txt"):
                with open(f"/home/uraninjo/Desktop/Çalışmalar/Comp/LABELS/{_hno}/results.txt", "a") as f:
                    f.write(f"{_tür} {_hno} {birads_predicted_label} {kompozisyon_predicted_label}\n")
            else:
                with open(f"/home/uraninjo/Desktop/Çalışmalar/Comp/LABELS/{_hno}/results.txt", "w") as f:
                    f.write(f"{_tür} {_hno} {birads_predicted_label} {kompozisyon_predicted_label}\n")

In [6]:
birads_model_path="/home/uraninjo/Desktop/Çalışmalar/Mergen1-Teknofest/Murat/checkpoints3/Kullanılacaklar/BIRADSPosta2_effb0_Murat_0.7095acc_50epochs.h5"
kompozisyon_model_path="/home/uraninjo/Desktop/Çalışmalar/Mergen1-Teknofest/Murat/checkpoints3/Kullanılacaklar/KompozisyonPosta2_0.7223acc_50epochs.h5"
birads_model=torch.load(birads_model_path)
kompozisyon_model=torch.load(kompozisyon_model_path)

test(birads_model, kompozisyon_model, dataloader)

LABELS dosyası zaten var
txt dosyası zaten var
LABELS dosyası zaten var
txt dosyası zaten var
LABELS dosyası zaten var
txt dosyası zaten var
LABELS dosyası zaten var
LABELS dosyası zaten var
txt dosyası zaten var
LABELS dosyası zaten var
txt dosyası zaten var
LABELS dosyası zaten var
txt dosyası zaten var
LABELS dosyası zaten var
LABELS dosyası zaten var
txt dosyası zaten var
LABELS dosyası zaten var
txt dosyası zaten var
LABELS dosyası zaten var
txt dosyası zaten var
LABELS dosyası zaten var
LABELS dosyası zaten var
txt dosyası zaten var
LABELS dosyası zaten var
txt dosyası zaten var
LABELS dosyası zaten var
txt dosyası zaten var
LABELS dosyası zaten var
LABELS dosyası zaten var
txt dosyası zaten var
LABELS dosyası zaten var
txt dosyası zaten var
LABELS dosyası zaten var
txt dosyası zaten var
LABELS dosyası zaten var
LABELS dosyası zaten var
txt dosyası zaten var
LABELS dosyası zaten var
txt dosyası zaten var
LABELS dosyası zaten var
txt dosyası zaten var
LABELS dosyası zaten var
LABE

## 400 Fotoğrafın BIRADS ve Meme Yoğunluğundan Çıkma Süresi --> 32.9sn

"/home/uraninjo/Desktop/Çalışmalar/Mergen1-Teknofest/Murat/checkpoints3/Kullanılacaklar/Eniyi_KompozitPosta12_Sinem_0.7896acc_50epochs.h5"
400-120 doğruluk oranı
-----
"/home/uraninjo/Desktop/Çalışmalar/Mergen1-Teknofest/Murat/checkpoints3/Kullanılacaklar/KompozisyonPosta2_0.7223acc_50epochs.h5"
400-306


"/home/uraninjo/Desktop/Çalışmalar/Mergen1-Teknofest/Murat/checkpoints3/Kullanılacaklar/BIRADSPosta2_effb0_Murat_0.7095acc_50epochs.h5"
400-269
----
"/home/uraninjo/Desktop/Çalışmalar/Mergen1-Teknofest/Murat/checkpoints3/Kullanılacaklar/eniyi_BIRADS_0.7818acc_35epochs.h5"
400-17